In [140]:
import sys
import os
sys.path.append('..')
sys.path.append('../src')

from src.utilsHVAE import *
from src.expression_set_generation import *
from src.model import HVAE
from src.symbol_library import generate_symbol_library
import torch
from src.Create_rate_law import create_rate_law, create_rnd_math
import pickle
from sklearn.decomposition import PCA

In [141]:
# Steps
# 1. generate rate laws and random math equations 
# 2. Take out all math equations which are found in the rate laws
# 3. Train HVAE to for the given symbols and paramters
# 4. Encode all rate laws and math functions
# 4.5 do PCA and transform the relevant data.
# 5. Train a support vector classifier (or any other kind of classifier) where rate laws are 1 and random math is -1
# 6. generate a random cordinate which is not in the original set and is classified by the classifier to be in class 1
# 7. Decode this value into a function.
# 8. see what it looks like    

In [142]:

symbols = ["-", "+", "/", "*", "^2", "^3", "^4", "^5", 'cos','sin','sqrt']
num_variables = 3
has_constants = True
num_expressions = 1000000
max_tree_height = 20

sy_lib = generate_symbol_library(num_variables, symbols, has_constants)
# sy_lib = generate_symbol_library(2, expr_config["symbols"], True)
so = {s["symbol"]: s for s in sy_lib}
HVAE.add_symbols(sy_lib)

In [143]:
fake_sim = list(create_rate_law("sim"))
fake_rev = list(create_rate_law("rev"))
fake_cplx = list(create_rate_law("cplx"))

fake_rate = fake_sim + fake_rev + fake_cplx
rate_class = np.ones(len(fake_rate))

fake_math = list(create_rnd_math())
math_class = -1 * np.ones(len(fake_math))



Unique expressions generated so far: 0
Unique expressions generated so far: 0
Unique expressions generated so far: 500
Unique expressions generated so far: 500
Unique expressions generated so far: 0
Unique expressions generated so far: 500
Unique expressions generated so far: 500
Unique expressions generated so far: 500
Unique expressions generated so far: 500
Unique expressions generated so far: 0
Unique expressions generated so far: 500
Unique expressions generated so far: 500
Unique expressions generated so far: 1000
Unique expressions generated so far: 1000
Unique expressions generated so far: 1500
Unique expressions generated so far: 1500
Unique expressions generated so far: 1500
Unique expressions generated so far: 1500
Unique expressions generated so far: 2000
Unique expressions generated so far: 2000
Unique expressions generated so far: 2000
Unique expressions generated so far: 2000
Unique expressions generated so far: 2500
Unique expressions generated so far: 2500


In [144]:
fake_math

['( ( B / sin ( cos ( C ) * ( A + B ) ) ) ) ^2',
 'B / ( D + sin ( C * ( ( ( sqrt ( ( sin ( ( C ) * B ) * C ) ) + ( ( C ) ) ^2 ) ) ^4 ) + A ) ) ^2',
 '( cos ( B / D / D - B ) )',
 'sqrt ( ( C + sin ( A ) / sqrt ( D ) ) / B )',
 'cos ( sqrt ( ( ( A ) - C ) ) )',
 'C / cos ( C / C - A )',
 '( cos ( D ) + D ) ^2 + C',
 'A * C + A - ( ( C ) ^2 ) ^4',
 '( C ) ^2 - B / C',
 'A / ( sqrt ( sin ( sin ( B ) ) ) + C )',
 '( C ) / D',
 '( ( ( ( B ) ^3 ) ) ^4 )',
 '( ( C ) ) ^4 - ( C * ( B ) ^2 - cos ( C ) )',
 '( ( ( ( sin ( B ) / A ) * B ) ^3 ) - A / C ) / C - ( C ) / sqrt ( C )',
 'B * sqrt ( ( ( ( cos ( B / C ) ) / C ) ^3 ) ^3 ) - D',
 'B / A * B',
 'C * ( D + C * ( B ) + D / A ) ^2',
 'D + B + D * C',
 '( A ) ^4 / A',
 'C / B / ( ( A ) + sin ( A / C ) ) * ( cos ( C - C ) - A )',
 'C * ( C - ( C ) / C ) ^4',
 '( D - C )',
 '( ( C ) ^2 * C )',
 'B + C / A',
 '( sqrt ( D ) )',
 'sqrt ( ( A - A - ( ( D ) ) ) ^3 + A )',
 '( ( C + B ) ) ^4',
 '( C ) * ( A - A + ( A - D * sin ( sqrt ( C ) - ( sin ( s

In [145]:
param_file = "../params/rate_training_3v_20_15k.pt"
model = torch.load(param_file)
model.add_symbols(sy_lib)

rate_token_list = [expr.split(" ") for expr in fake_rate]
math_token_list = [expr.split(" ") for expr in fake_math]

rate_tree_list = [tokens_to_tree(token,so) for token in rate_token_list]
math_tree_list = [tokens_to_tree(token,so) for token in math_token_list]

rate_Btree_list = [create_batch([tree]) for tree in rate_tree_list]
math_Btree_list = [create_batch([tree]) for tree in math_tree_list]

rate_l_list = [model.encode(treeB) for treeB in rate_Btree_list]
math_l_list = [model.encode(treeB) for treeB in math_Btree_list]


rate_decoded = [str(model.decode(l[0])[0]) for l in rate_l_list]
math_decoded = [str(model.decode(l[0])[0]) for l in math_l_list]

In [150]:
for i in range(len(math_decoded)):
    out = fake_math[i].replace(" ", "")
    print(f"O:{out} p:{math_decoded[i]}\n")

O:((B/sin(cos(C)*(A+B))))^2 p:(B/sin(cos(C)*cos(A)))^2

O:B/(D+sin(C*(((sqrt((sin((C)*B)*C))+((C))^2))^4)+A))^2 p:B/(D+sin(C*sin(sin(C+A)+A)+A))^2

O:(cos(B/D/D-B)) p:cos(B/D/D-B)

O:sqrt((C+sin(A)/sqrt(D))/B) p:sqrt((C+sin(A)/sqrt(D))/B)

O:cos(sqrt(((A)-C))) p:cos(sqrt(A-C))

O:C/cos(C/C-A) p:C/cos(C/C-A)

O:(cos(D)+D)^2+C p:(cos(D)+D)^2+C

O:A*C+A-((C)^2)^4 p:A*C+A-((C)^2)^4

O:(C)^2-B/C p:(C)^2-B/C

O:A/(sqrt(sin(sin(B)))+C) p:A/(sqrt(sin(cos(B)))+C)

O:(C)/D p:C/D

O:((((B)^3))^4) p:((B)^3)^4

O:((C))^4-(C*(B)^2-cos(C)) p:(C)^4-C*(B)^2*cos(C)

O:((((sin(B)/A)*B)^3)-A/C)/C-(C)/sqrt(C) p:(cos(B*C/B*A)-A/C)/C-C/sqrt(C)

O:B*sqrt((((cos(B/C))/C)^3)^3)-D p:B*sqrt(((B/C*C/C)^3)^3)-D

O:B/A*B p:B/A*B

O:C*(D+C*(B)+D/A)^2 p:C*(D+C*B+D/A)^2

O:D+B+D*C p:D+B+D*C

O:(A)^4/A p:(A)^4/A

O:C/B/((A)+sin(A/C))*(cos(C-C)-A) p:C/B/(A+A*D)*(cos(C-C)-A)

O:C*(C-(C)/C)^4 p:C*(C-C/C)^4

O:(D-C) p:D-C

O:((C)^2*C) p:(C)^2*C

O:B+C/A p:B+C/A

O:(sqrt(D)) p:sqrt(D)

O:sqrt((A-A-((D)))^3+A) p:sqrt((A-A-D)^

In [152]:
str(model.decode(rate_l_list[0][0])[0])

'C*(B)^2'

In [153]:
rate_latent = np.array([rate_l_list[i][0].detach().numpy()[0][0] for i in range(len(rate_l_list))])
math_latent = np.array([math_l_list[i][0].detach().numpy()[0][0] for i in range(len(math_l_list))])

In [154]:
nums = range(min(rate_latent.shape))
for num in nums:
  pca = PCA(n_components=num)
  pca.fit(np.append(rate_latent, math_latent, axis=0))
  if np.sum(pca.explained_variance_ratio_)>0.99:
    n = num
    print(n)
    break

46


In [155]:
rate_pca = pca.transform(rate_latent)
math_pca = pca.transform(math_latent)

In [200]:
with open("all_simple.pickle", "rb") as f:
    simple_rate = pickle.load(f)
with open("all_rev.pickle", "rb") as f:
    rev_rate = pickle.load(f)
with open("all_cplx.pickle", "rb") as f:
    cplx_rate = pickle.load(f)


In [201]:
t_rate_data = list(simple_rate) + list(rev_rate) + list(cplx_rate)

t_rate_token_list = [expr.split(" ") for expr in t_rate_data]
t_rate_tree_list = [tokens_to_tree(token,so) for token in t_rate_token_list]
t_rate_Btree_list = [create_batch([tree]) for tree in t_rate_tree_list]
t_rate_l_list = [model.encode(treeB) for treeB in t_rate_Btree_list]
t_rate_latent = np.array([t_rate_l_list[i][0].detach().numpy()[0][0] for i in range(len(t_rate_l_list))])
# t_rate_pca = pca.transform(t_rate_latent)


In [ ]:
nums = range(min(t_rate_latent.shape))
for num in nums:
  pca = PCA(n_components=num)
  pca.fit(np.append(rate_latent, math_latent, axis=0))
  if np.sum(pca.explained_variance_ratio_)>0.99:
    n = num
    print(n)
    break

In [156]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
scaled_pca_data = std_scaler.fit_transform(np.append(rate_pca,math_pca,axis=0))
# scaled_math_pca = std_scaler.fit_transform(math_pca)

In [157]:
from sklearn.svm import SVC
classifier = SVC(probability=True)
classifier.fit(np.append(rate_pca,math_pca,axis=0), np.append(rate_class, math_class))
classifier.fit(scaled_pca_data, np.append(rate_class, math_class))


SVC(probability=True)

In [158]:
maxv = 0
minv = 0 
for a in np.append(rate_pca,math_pca,axis=0):
    for b in a:
        maxv = max(maxv, b)
        minv = min(minv, b)
print(maxv, minv)

4.7922673 -3.497498


In [159]:
classifier.n_features_in_

46

In [183]:
# ! Scaling  the data breaks the decoding... Undo this...
flag = True
while flag:
    testpoint= (2*np.random.rand(1,classifier.n_features_in_))-1
    if classifier.predict_proba(testpoint)[0][1]>  0.95:
        flag = False
testpoint = std_scaler.inverse_transform(testpoint)
testpoint = torch.from_numpy(pca.inverse_transform(testpoint).reshape(1,1,128)).to(torch.float32)
f_eqn = model.decode(testpoint)
str(f_eqn[0])





'C*A/(D*D+A)'

In [193]:
testpoint= (2*np.random.rand(1,46))-1
print(testpoint, classifier.predict_proba(testpoint)) 
# # classifier.predict(testpoint)
# testpoint = testpoint.reshape(1,1,38)
# testpoint.size(0)


[[-0.40248505 -0.890442    0.46867664 -0.33067859  0.88544326 -0.59641705
   0.49206512 -0.78548127 -0.47178607  0.45464183  0.09536801 -0.01532995
  -0.63455977  0.60894281 -0.61336419 -0.81967678 -0.19625003  0.99356697
   0.21782233  0.04486985 -0.16536518 -0.91424797 -0.16541051  0.29719961
   0.90020043 -0.87311818  0.20934247  0.50208799  0.7257583  -0.92319339
  -0.81361846  0.72726194 -0.63640185 -0.37443824 -0.77288219 -0.64233737
  -0.57104464  0.2210278  -0.92893848  0.38346743  0.82022129  0.80916939
  -0.09084725  0.11912092 -0.15459697  0.98327294]] [[0.20714953 0.79285047]]
